In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

In [2]:
%run preprocessing.ipynb
%run temporal_preprocess.ipynb
%run graph.ipynb

100%|██████████| 6/6 [09:21<00:00, 93.53s/it] 


                       Feature  Importance
0                       ged_sb    0.109912
1               decay_ged_ns_1    0.093771
2      sptime_dist_k001_ged_ns    0.056070
3       splag_1_decay_ged_sb_1    0.035770
4      sptime_dist_k001_ged_os    0.035527
5           wdi_nv_agr_totl_kd    0.032515
6       sptime_dist_k10_ged_ns    0.026288
7       sptime_dist_k10_ged_os    0.026096
8        sptime_dist_k1_ged_ns    0.022685
9              spei_48_detrend    0.022117
10           spei1_gsm_detrend    0.018739
11  ged_sb_decay_12_time_since    0.018219
12       mov_avg_6_ged_best_sb    0.018188
13       mov_sum_6_ged_best_sb    0.016237
14                treelag_1_ns    0.014685
15                   ged_gte_1    0.014666
16                treelag_2_os    0.014348
17                treelag_2_ns    0.014340
18                treelag_1_os    0.014174
19             spei1_gs_prev10    0.012800
20      sptime_dist_k10_ged_sb    0.012221
21       sptime_dist_k1_ged_sb    0.012162
22        s

In [3]:
data_ns, adj_matrix_ns, conv_layer_ns = create_graph_pyg_ns(df_filtered, num_neighbors=4, hidden_channels=64)

In [14]:
df_filtered

,ged_sb,decay_ged_ns_1,sptime_dist_k001_ged_ns,splag_1_decay_ged_sb_1,sptime_dist_k001_ged_os,wdi_nv_agr_totl_kd,sptime_dist_k10_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k1_ged_ns,spei_48_detrend,...,ged_sb_tlag_12,ged_sb_tlag_5,treelag_1_sb,count_moder_drought_prev10,treelag_2_sb,ged_sb_tlag_10,tlag_12_harvarea_maincrops,tlag_12_crop_sum,date,priogrid_gid
0,-0.026241,-0.165654,4.738637,-0.371313,9.639623,-0.368149,6.738254,6.532327,6.400377,0.885765,...,-0.028663,-0.027348,-0.418114,-0.891808,-0.170367,-0.028344,-0.388428,-0.333487,2016-01-01,-3.086626
1,-0.026241,-0.165654,-0.475191,-0.371299,0.392014,-0.368149,4.768209,4.485963,4.577182,1.637610,...,-0.028663,-0.027348,-0.341964,-0.891808,-0.167622,-0.028344,-0.453037,-0.310987,2016-01-01,-2.441018
2,-0.026241,-0.165654,-0.419315,-0.371295,0.420701,-0.368149,4.792157,4.459127,4.558453,1.637610,...,-0.028663,-0.027348,-0.341295,0.022680,-0.167596,-0.028344,-0.068737,0.304655,2016-01-01,-2.440981
3,-0.026241,-0.165654,-0.333919,-0.371299,0.504226,-0.368149,4.816913,4.433556,4.540879,1.180225,...,-0.028663,-0.027348,-0.340648,0.632338,-0.167570,-0.028344,0.081107,2.085527,2016-01-01,-2.440943
4,-0.026241,-0.165654,-0.494762,-0.371303,0.315186,-0.368149,4.650759,4.457748,4.549976,1.033693,...,-0.028663,-0.027348,-0.339685,-0.891808,-0.167533,-0.028344,-0.451336,-0.337173,2016-01-01,-2.414135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,-0.026241,-0.165654,2.320298,-0.371311,-0.447833,-0.624533,0.158557,1.087421,0.405020,0.082860,...,-0.028663,-0.027348,0.457276,-0.891808,-0.112974,-0.028344,-0.491396,-0.367803,2021-10-01,1.711159
1048796,-0.026241,-0.165654,0.995333,-0.371311,0.284241,2.491172,-0.470458,0.283821,-0.273830,0.082860,...,-0.028663,-0.027348,0.622447,-0.891808,-0.087101,-0.028344,-0.491396,-0.367803,2021-10-01,1.711571
1048797,-0.026241,-0.165654,0.908653,-0.371311,0.374058,2.491172,-0.512859,0.220832,-0.316462,0.082860,...,-0.028663,-0.027348,0.451083,-0.891808,-0.104159,-0.028344,-0.491396,-0.367803,2021-10-01,1.711609
1048798,-0.026241,-0.165654,0.767960,-0.371311,0.695895,2.491172,-0.584801,0.102800,-0.385566,0.082860,...,-0.028663,-0.027348,0.471825,-0.891808,-0.100443,-0.028344,-0.491396,-0.367803,2021-10-01,1.711683


In [4]:
class HybridGCNTransformer(nn.Module):
    def __init__(self, gcn_model, transformer_model, gcn_output_dim, transformer_output_dim, final_output_dim):
        super(HybridGCNTransformer, self).__init__()
        self.gcn = gcn_model
        self.transformer = transformer_model
        # Adjust the dimensions according to the last layer's output of GCN and Transformer
        self.fc1 = nn.Linear(gcn_output_dim + transformer_output_dim, 128)
        self.fc2 = nn.Linear(128, final_output_dim)
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()

    def forward(self, x_graph, adj_matrix, x_time, dates):
        # Forward pass through GCN
        gcn_output = self.gcn(x_graph, adj_matrix)
        # Assuming gcn_output and transformer_output are already in the proper shape to be concatenated
        # Forward pass through Transformer
        transformer_output = self.transformer(x_time, dates)
        # Concatenate the outputs from GCN and Transformer
        combined_features = torch.cat((gcn_output, transformer_output), dim=1)
        # Fully connected layers
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [17]:
class GraphDataset(Dataset):
    def __init__(self, data, adj_matrix):
        self.data = data
        self.adj_matrix = adj_matrix

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        gid = list(self.data.keys())[idx]
        # Exclude first column and last column
        
        target = np.array(self.data[gid][:, -1], dtype=np.float32)
        return torch.tensor(features), self.adj_matrix, torch.tensor(target)
    

class TransformerDataset(Dataset):
    def __init__(self, data, date_encodings):
        self.data = data
        self.date_encodings = date_encodings

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        gid = list(self.data.keys())[idx]
        features = np.array(self.data[gid][:, :-2], dtype=np.float32)  # Exclude the last two columns if last is target and second last is date
        target = np.array(self.data[gid][:, -1], dtype=np.float32)
        dates = self.date_encodings[gid]
        return torch.tensor(features), torch.tensor(dates), torch.tensor(target)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x, dates):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TemporalTransformerEncoder(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dim_feedforward=512, dropout=0.1):
        super(TemporalTransformerEncoder, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model // 2)
        self.pos_encoder = PositionalEncoding(d_model // 2, dropout=dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model // 2, nhead=nhead // 2, dim_feedforward=dim_feedforward // 2, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_layer = nn.Linear(d_model // 2, 1)

    def forward(self, x, dates):
        x = self.input_embedding(x)
        x = self.pos_encoder(x, dates)
        x = self.transformer_encoder(x)
        output = self.output_layer(x)
        return output.squeeze(-1)
    
# Graph Convolution Layer
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.Tensor(output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        support = torch.matmul(x, self.weight)
        output = torch.sparse.mm(adj, support)
        output = output + self.bias
        return F.relu(output)

# GCN Model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim1)
        self.gc2 = GraphConvolution(hidden_dim1, hidden_dim2)
        self.gc3 = GraphConvolution(hidden_dim2, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.norm1 = nn.LayerNorm(hidden_dim1)
        self.norm2 = nn.LayerNorm(hidden_dim2)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.norm1(x)
        
        x = self.gc2(x, adj)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.norm2(x)
        
        x = self.gc3(x, adj)
        return x


In [18]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Custom Dataset that provides both graph and temporal data for each item
class HybridDataset(Dataset):
    def __init__(self, graph_data, temporal_data, temporal_date_encodings):
        self.graph_data = graph_data  # data for GCN
        self.temporal_data = temporal_data  # data for Transformer
        self.temporal_date_encodings = temporal_date_encodings

    def __len__(self):
        return len(self.graph_data)

    def __getitem__(self, idx):
        # Graph Data
        x_graph, adj_matrix, y_graph = self.graph_data[idx]
        
        # Temporal Data
        x_temporal, dates, y_temporal = self.temporal_data[idx]
        
        return x_graph, adj_matrix, x_temporal, dates, y_graph  # Assuming y_graph == y_temporal

# Example of DataLoader setup
graph_dataset = GraphDataset(data_ns, adj_matrix_ns)
temporal_dataset = TransformerDataset(transformer_input, date_encodings)
hybrid_dataset = HybridDataset(graph_dataset, temporal_dataset, date_encodings)
train_set, val_set = train_test_split(range(len(hybrid_dataset)), test_size=0.2, random_state=42)
train_loader = DataLoader([hybrid_dataset[i] for i in train_set], batch_size=64, shuffle=True)
val_loader = DataLoader([hybrid_dataset[i] for i in val_set], batch_size=64, shuffle=False)

# Initialize the hybrid model
gcn_model = GCN(input_dim=data_ns.x.size(1), hidden_dim1=256, hidden_dim2=256, output_dim=1)
transformer_model = TemporalTransformerEncoder(input_size=len(top_50_features), d_model=12, nhead=4, num_layers=16)
hybrid_model = HybridGCNTransformer(gcn_model, transformer_model, 256, 6, 1)
optimizer = torch.optim.Adam(hybrid_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_losses = []
val_losses = []
val_r2_scores = []

def r_squared(y_true, y_pred):
    ss_res = ((y_true - y_pred) ** 2).sum()
    ss_tot = ((y_true - y_true.mean()) ** 2).sum()
    r2 = 1 - ss_res / ss_tot
    return r2.item()

for epoch in range(10):  # Number of epochs
    hybrid_model.train()
    total_train_loss = 0

    for x_graph, adj_matrix, x_temporal, dates, y in train_loader:
        optimizer.zero_grad()

        # Run the hybrid model forward pass
        output = hybrid_model(x_graph, adj_matrix, x_temporal, dates)
        
        # Compute loss
        loss = criterion(output, y)
        total_train_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation phase
    hybrid_model.eval()
    total_val_loss = 0
    total_r2 = 0
    with torch.no_grad():
        for x_graph, adj_matrix, x_temporal, dates, y in val_loader:
            output = hybrid_model(x_graph, adj_matrix, x_temporal, dates)
            loss = criterion(output, y)
            total_val_loss += loss.item()
            total_r2 += r_squared(y, output)  # Ensure r_squared is compatible with your data

    avg_val_loss = total_val_loss / len(val_loader)
    avg_r2 = total_r2 / len(val_loader)
    val_losses.append(avg_val_loss)
    val_r2_scores.append(avg_r2)

    print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val R^2: {avg_r2:.4f}')

    # # Optional: save the model if validation loss improves
    # if avg_val_loss < some_threshold:
    #     torch.save(hybrid_model.state_dict(), 'model_path.pth')
    #     print(f"Model saved at 'model_path.pth'")



IndexError: too many indices for tensor of dimension 1